## <center> Leitura e limpeza dos dados de artigos do Medium </center>
<img src="data_cleaning.png" width="300">

Neste notebook será realizada a limpeza dos dados dos artigos coletados no Medium (colocar link para o notebook de coleta).

### Coleta dos dados
Os dados foram coletados no dia 03/02/2021 a partir de 7 tags que mais se relacionam com o tema inteligência artificial, são elas: 'AI', 'artificial-intelligence', 'data', 'data-science', 'deep-learning', 'machine-learning', 'neural-networks'
Foram extraídas as publicações diárias no período de 01/01/2020 a 31/12/2020.

### Estrutura dos dados

* Title - título do artigo no card de extração
* Subtitle - subtítulo do artigo no card de extração
* Image (yes/no)- indica se o artigo possui uma imagem de preview no card de extração
* Author - autor do artigo
* Publication - título da publicação à qual o artigo está vinculada, para o caso de artigos independentes está marcado como "No * * publication"
* Year - Month - Day - data em que o artigo foi publicado
* Tag - Tag à qual o artigo foi vinculado, lembrando que um artigo pode ser vinculado à até 5 tags
* Reading Time- tempo de leitura do artigo
* Claps - número de aplausos que o artigo recebeu até a data de coleta dos dados
* Comment (yes/no) - indica se a entrda é um comentário em um outro artigo
* Story Url - link para o artigo
* Author URL - link para a homepage do autor


### Carregando libraries necessárias

In [1]:
import os 
import glob
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Realizando a leitura dos arquivos .csv e criando o dataframe para limpeza

In [2]:
paths = glob.glob("D:\\Documentos\\Projetos\\Sistemas_de_recomendacao\\Scrap_medium\\by_tag\\TAG_SCRAPES/*")
list_ = []
for path in paths:
    file_list = glob.glob(path + "/*.csv")
    for file_ in file_list:
        df = pd.read_csv(file_,index_col=None, header=0, sep=';')
        list_.append(df)

artigos_raw = pd.concat(list_)
artigos_raw.shape

(177714, 14)

In [3]:
artigos_raw.head()
#df_trat_2 = df_trat.drop_duplicates(subset=['Title','Subtitle', 'Author', 'Publication', 'Year', 'Month', 'Day', 'url'])

,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Tag,Reading_Time,Claps,Comment,url,Author_url
0,How to go from BayesTheorem to Bayesian Inference,NaN,1,JimSpark,Towards Data Science,2020,10,1,AI,10,68,0,https://towardsdatascience.com/how-to-go-from-...,https://towardsdatascience.com/@jimip6c12?sour...
1,Autoencoders: Overview of Research and Applica...,NaN,1,Branislav Holl nder,Towards Data Science,2020,10,1,AI,10,129,0,https://towardsdatascience.com/autoencoders-ov...,https://towardsdatascience.com/@branislav.holl...
2,Regarding SingularityNET and the Recent KuCoin...,NaN,1,Ben Goertzel,SingularityNET,2020,10,1,AI,8,320,0,https://blog.singularitynet.io/regarding-singu...,https://blog.singularitynet.io/@ben_90344?sour...
3,Understand Machine Learning with One Article,Lets dive into one of the most mind-blowing an...,1,Julian Herrera,Towards Data Science,2020,10,1,AI,11,67,0,https://towardsdatascience.com/understand-mach...,https://towardsdatascience.com/@julianh?source...
4,The End of the World,Below is a conversation I had with OpenAIs GPT...,1,Kirk Ouimet,NaN,2020,10,1,AI,5,60,0,https://medium.com/@kirkouimet/the-end-of-the-...,https://medium.com/@kirkouimet?source=tag_arch...


In [4]:
artigos_raw.dtypes

Title           object
Subtitle        object
Image            int64
Author          object
Publication     object
Year             int64
Month            int64
Day              int64
Tag             object
Reading_Time     int64
Claps           object
Comment          int64
url             object
Author_url      object
dtype: object

### Reformatando o campo "Claps" para float
No campo "Claps" os casos maiores que 999 são abreviados com "K", por exemplo, 2000 é escrito como 2K. Aqui vamos remover o "K", converter o campo de string para float e multiplicar por 1000.

In [5]:
numeric_claps = []
for x in artigos_raw.Claps:
    if "K" in str([x]):
        numeric_claps.append(float(x[:-1])*1000)
    else:
        numeric_claps.append(x)
artigos_raw["Claps"] = numeric_claps
artigos_raw["Claps"] = pd.to_numeric(artigos_raw["Claps"])
print("Clap dtype: ", artigos_raw.dtypes["Claps"])

Clap dtype:  float64


In [6]:
artigos_raw.dtypes

Title            object
Subtitle         object
Image             int64
Author           object
Publication      object
Year              int64
Month             int64
Day               int64
Tag              object
Reading_Time      int64
Claps           float64
Comment           int64
url              object
Author_url       object
dtype: object

### Limpando Urls.

In [7]:
#Antes
for i in range(3):
    print(artigos_raw.Author_url.values[i])

https://towardsdatascience.com/@jimip6c12?source=tag_archive---------0-----------------------
https://towardsdatascience.com/@branislav.hollander?source=tag_archive---------1-----------------------
https://blog.singularitynet.io/@ben_90344?source=tag_archive---------2-----------------------


In [8]:
artigos_raw.url = artigos_raw.url.str.split("?", expand=True)
artigos_raw.Author_url = artigos_raw.Author_url.str.split("?", expand=True)

In [9]:
#Depois
for i in range(3):
    print(artigos_raw.Author_url.values[i])

https://towardsdatascience.com/@jimip6c12
https://towardsdatascience.com/@branislav.hollander
https://blog.singularitynet.io/@ben_90344


### Checando nulos nos dados
#### Quantidade de nulos por coluna
Só existem valores nulos nos campos Title, Subtitle, Author e Publication. **NaNs na coluna Publication é porque nem todos os artigos são publicados através de um grupo.**

In [10]:
print("Number of NaNs")
artigos_raw.isna().sum()
print()
print("Total Entries:  ", artigos_raw.shape[0])

Number of NaNs


Title           13063
Subtitle        71285
Image               0
Author            115
Publication     84496
Year                0
Month               0
Day                 0
Tag                 0
Reading_Time        0
Claps               0
Comment             0
url                 0
Author_url        115
dtype: int64


Total Entries:   177714


### NaN nos Títulos e Subtítulo
Alguma vezes, quando se raspa os dados, os títulos estão em formatos estranhos. O resultado é que, **alguns títulos são retornados como subtítulo.**

Abaixo está uma breve análise de nulos no Título/Subtítulo. **Neste caso, escolheu-se excluir artigos sem títulos e sem autor.**

In [11]:
print("Total de registros com título nulo: ", artigos_raw[artigos_raw.Title.isnull()].shape[0])

print("Total de registros com título nulo mas com subtítulo: ",artigos_raw[(artigos_raw.Title.isnull() & artigos_raw.Subtitle.notnull())].shape[0])

print("Registros sem título e subtítulo: ", artigos_raw[(artigos_raw.Title.isnull() & artigos_raw.Subtitle.isnull())].shape[0])

Total de registros com título nulo:  13063
Total de registros com título nulo mas com subtítulo:  4652
Registros sem título e subtítulo:  8411


### Removendo títulos nulos

In [12]:
df_trat = artigos_raw.dropna(subset=["Title"])
df_trat = df_trat.dropna(subset=["Author"])

In [13]:
df_trat.shape

(164537, 14)

### Quantos nulos restaram

In [14]:
print("Number of NaNs")
df_trat.isna().sum()
print()
print("Total Entries:  ", df_trat.shape[0])

Number of NaNs


Title               0
Subtitle        62821
Image               0
Author              0
Publication     78871
Year                0
Month               0
Day                 0
Tag                 0
Reading_Time        0
Claps               0
Comment             0
url                 0
Author_url          0
dtype: int64


Total Entries:   164537


### Preenchendo nulos nos campos "Subtitle" e "Publication"
Para estes campo, como nem sempre pode ser obrigatório um artigo possuir uma publicação ou subtítulo, resolveu-se preencher com "No subtitle" e "No publication" os valores nulos para "Subtitle" e "Publication", respectivamente.

In [15]:
df_trat["Subtitle"].fillna("No subtitle", inplace = True)
df_trat["Publication"].fillna("No publication", inplace = True)

In [16]:
print("Number of NaNs")
df_trat.isna().sum()
print()
print("Total Entries:  ", df_trat.shape[0])

Number of NaNs


Title           0
Subtitle        0
Image           0
Author          0
Publication     0
Year            0
Month           0
Day             0
Tag             0
Reading_Time    0
Claps           0
Comment         0
url             0
Author_url      0
dtype: int64


Total Entries:   164537


### Removendo artigos que porventura tenham sido publicados por mais de um autor e salvando os dados para uso posterior

In [17]:
df_trat_2 = df_trat.drop_duplicates(subset=['Title','Subtitle', 'Publication', 'Year', 'Month', 'url', 'Tag'])

In [18]:
df_trat_2.shape

(164537, 14)

In [19]:
df_trat_2.to_csv('D:/Documentos/Projetos/Sistemas_de_recomendacao/Scrap_medium/by_tag/recommender_csv/medium_articles_clean.csv', index=False)